<a href="https://colab.research.google.com/github/xbarto0c/MPC-MLF/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop # RootMeanSquare Error
import numpy as np
import random


!wget -O kafka_Metamorphosis.txt https://www.gutenberg.org/files/5200/5200-0.txt

--2023-04-06 06:41:46--  https://www.gutenberg.org/files/5200/5200-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142017 (139K) [text/plain]
Saving to: ‘kafka_Metamorphosis.txt’

kafka_Metamorphosis 100%[===================>] 138.69K  --.-KB/s    in 0.04s   

2023-04-06 06:41:46 (3.09 MB/s) - ‘kafka_Metamorphosis.txt’ saved [142017/142017]



In [28]:
text = open('kafka_Metamorphosis.txt', 'r').read().lower();
print('text length: ',len(text));
print(text[:2020]);

text length:  138408
﻿the project gutenberg ebook of metamorphosis, by franz kafka

this ebook is for the use of anyone anywhere in the united states and
most other parts of the world at no cost and with almost no restrictions
whatsoever. you may copy it, give it away or re-use it under the terms
of the project gutenberg license included with this ebook or online at
www.gutenberg.org. if you are not located in the united states, you
will have to check the laws of the country where you are located before
using this ebook.

** this is a copyrighted project gutenberg ebook, details below **
**     please follow the copyright guidelines in this file.     **

title: metamorphosis

author: franz kafka

translator: david wyllie

release date: may 13, 2002 [ebook #5200]
[most recently updated: may 20, 2012]

language: english

character set encoding: utf-8

copyright (c) 2002 by david wyllie.

*** start of the project gutenberg ebook metamorphosis ***




metamorphosis

by franz kafka

transla

In [29]:
chars = sorted(list(set(text)));
print('Num of chars: ', len(chars));

Num of chars:  62


In [30]:
char_indices = dict((c, i) for i, c in enumerate(chars));
indices_char = dict((i, c) for i, c in enumerate(chars));

In [31]:
# Dělení textu na části po 40 znacích -> učení se po frázích
maxlen = 40;
step = 3; # Krokování částí
sentences = [];
next_chars = []; # Další predikované znaky

for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i:i + maxlen]);
  next_chars.append(text[i + maxlen]);

print('nb sentences: ', len(sentences));
print(sentences[:3]);
print(next_chars[:3]);

nb sentences:  46123
['\ufeffthe project gutenberg ebook of metamorp', 'e project gutenberg ebook of metamorphos', 'roject gutenberg ebook of metamorphosis,']
['h', 'i', ' ']


In [32]:
# One-hot encoding
x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool);
y = np.zeros((len(sentences), len(chars)), dtype = np.bool);
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1;
    y[i, char_indices[next_chars[i]]] = 1;
print(x[:3]);
print(y[:3]);

<ipython-input-32-e1d844c42748>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool);
<ipython-input-32-e1d844c42748>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype = np.bool);


[[[False False False ... False False  True]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False  True False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]]
[[False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False Fals

In [33]:
model = Sequential();
model.add(LSTM(254, input_shape = (maxlen, len(chars))));
model.add(Dense(10*len(chars)));
model.add(Dense(len(chars)));
model.add(Activation('softmax'));

In [34]:
model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(learning_rate = 0.01));

In [35]:
model.fit(x, y, batch_size = 128, epochs = 30);

Epoch 1/30
361/361 [==============================] - 4s 8ms/step - loss: 4.1816
Epoch 2/30
361/361 [==============================] - 3s 7ms/step - loss: 2.0216
Epoch 3/30
361/361 [==============================] - 3s 8ms/step - loss: 1.7609
Epoch 4/30
361/361 [==============================] - 3s 8ms/step - loss: 1.5985
Epoch 5/30
361/361 [==============================] - 3s 7ms/step - loss: 1.4789
Epoch 6/30
361/361 [==============================] - 3s 7ms/step - loss: 1.3714
Epoch 7/30
361/361 [==============================] - 3s 7ms/step - loss: 1.2833
Epoch 8/30
361/361 [==============================] - 3s 8ms/step - loss: 1.1951
Epoch 9/30
361/361 [==============================] - 3s 8ms/step - loss: 1.1047
Epoch 10/30
361/361 [==============================] - 3s 7ms/step - loss: 1.0263
Epoch 11/30
361/361 [==============================] - 3s 7ms/step - loss: 0.9553
Epoch 12/30
361/361 [==============================] - 3s 8ms/step - loss: 0.8891
Epoch 13/30
361/361 [====

In [47]:
# Vzorkování indexu z pole pravděpodobností
def sample(preds, temperature = 0.1):
  preds = np.asarray(preds).astype('float64');
  preds = np.log(preds) / temperature;
  exp_preds = np.exp(preds);
  preds = exp_preds / np.sum(exp_preds);
  probas = np.random.multinomial(1, preds, 1);
  return np.argmax(probas);
# Vybere pomocí pomocné funkce znak a sestaví větu
def generate_text(sentence, length, diversity):
  generated = '';
  generated += sentence;
  for i in range(length):
    x_pred = np.zeros((1, maxlen, len(chars)));
    for t, char in enumerate(sentence):
      x_pred[0, t, char_indices[char]] = 1.;
    
    preds = model.predict(x_pred, verbose = 0)[0];
    next_index = sample(preds, diversity);
    next_char = indices_char[next_index];

    generated += next_char;
    sentence = sentence[1:] + next_char;
  return generated;

In [50]:
text = "save the world from devastation, reunite"; # The imput to be completed, MUST be 40 chars long
       
sentence = text[0: maxlen];

print(generate_text(sentence, 30, 0.2));

save the world from devastation, reunited states complaintudelare must
